In [11]:
!pip install -q transformers accelerate datasets peft

In [12]:
# Load required libraries
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType

In [13]:
# load json dataset for finetuning 
encouraging_dataset = load_dataset("json", data_files="/kaggle/input/personality-finetuning-inputs/career_encouraging.jsonl", split="train")
critical_dataset = load_dataset("json", data_files="/kaggle/input/personality-finetuning-inputs/career_critical.jsonl", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [15]:
# Load tokenizer and define format prompt 
base_model_name = "Qwen/Qwen3-0.6B-Base"
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)

def format_prompt(example, tokenizer):
    full_prompt = f"### Instruction:\n{example['instruction']}\n\n### Response:\n{example['response']}"
    tokenized = tokenizer(
        full_prompt,
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    tokenized["labels"] = tokenized["input_ids"]
    return tokenized

tokenizer_config.json:   0%|          | 0.00/9.68k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [16]:
# Tokenize the encouraging dataset
tokenized_encouraging_dataset = encouraging_dataset.map(
    format_prompt,
    fn_kwargs={"tokenizer": tokenizer},
    remove_columns=encouraging_dataset.column_names
)

# Tokenize the critical dataset
tokenized_critical_dataset = critical_dataset.map(
    format_prompt,
    fn_kwargs={"tokenizer": tokenizer},
    remove_columns=critical_dataset.column_names
)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [17]:
# Configure trainer and train 

def train_lora_model(base_model_name, tokenized_dataset, persona_label):

    print(f"Training model for persona: {persona_label.upper()}")

    # Define save path
    output_dir = f"/kaggle/working/qwen-lora-career-{persona_label}"
    
    # Load base model
    model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        device_map="auto",
        torch_dtype="auto",
        trust_remote_code=True
    )
    
    # Disable cache for training 
    model.config.use_cache = False 

    # Config lora 
    lora_config = LoraConfig(
        r=8,
        lora_alpha=32,
        lora_dropout=0.05,
        bias="none",
        task_type=TaskType.CAUSAL_LM,
        target_modules=["q_proj", "v_proj"]
    )

    # Apply lora 
    model = get_peft_model(model, lora_config)

    # Print trainable params 
    print(f"Trainable parmams for persona: {persona_label.upper()}")
    model.print_trainable_parameters()

    # Training config 
    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,
        learning_rate=2e-4,
        num_train_epochs=3,
        logging_steps=10,
        fp16=True,
        save_strategy="epoch",
        save_total_limit=1,
        report_to="none"
    )

    # Trainer 
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
        tokenizer=tokenizer,
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
    )

    trainer.train()
    
    # Save adapter and tokenizer
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    print(f"Saved model for persona: {persona_label.upper()}")

In [ ]:
# Lora finetuning for encouraing and critical personas 
train_lora_model(base_model_name = base_model_name,
                 tokenized_dataset = tokenized_encouraging_dataset, 
                 persona_label = "encouraging")
train_lora_model(base_model_name = base_model_name,
                 tokenized_dataset = tokenized_critical_dataset, 
                 persona_label = "critical")

In [ ]:
# Data compression 
import shutil

# Compress the trained encouraging model
shutil.make_archive("/kaggle/working/qwen-lora-career-encouraging", 'zip', "/kaggle/working/qwen-lora-career-encouraging")

# Compress the trained critical model
shutil.make_archive("/kaggle/working/qwen-lora-career-critical", 'zip', "/kaggle/working/qwen-lora-career-critical")


In [21]:
from IPython.display import FileLink

# Create download links
display(FileLink('qwen-lora-career-encouraging.zip'))
display(FileLink('qwen-lora-career-critical.zip'))

/kaggle/working/qwen-lora-career-encouraging.zip

/kaggle/working/qwen-lora-career-critical.zip